In [ ]:
import numpy as np
import pickle  # 用 pickle 替代 cPickle
import gym
np.bool8 = np.bool_

In [ ]:
import gym

In [ ]:
env = gym.make("CliffWalking-v0")
observation, _ = env.reset()

In [ ]:
from torch.distributions import Categorical

In [ ]:
from torch import nn
import torch

In [ ]:
class MLP(nn.Module):
    
    
    def __init__(self,output_dim):
        super().__init__()
        self.embeddings = nn.Embedding(48,200)
        self.linear2 = nn.Linear(200,output_dim,bias=False)
        
       # Xavier 初始化 + 偏置归零
        nn.init.xavier_normal_(self.embeddings.weight)
        nn.init.xavier_normal_(self.linear2.weight)

        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self,idxs):
        x = self.embeddings(idxs)
        x = self.relu(x)
        x = self.linear2(x)
        return self.softmax(x)
    
#     def sample_action(self,x):
# #         with torch.no_grad():
#                 # 添加探索机制
#         if np.random.random() < 0.8:
#             action = np.random.randint(0, 4)
#             probs = self.forward(x).squeeze()
# #             print(probs.shape,action)
#             return torch.LongTensor([action]), torch.log(probs[action]+1e-8).unsqueeze(dim=-1)
        
#         prob = self.forward(x)
#         dist = Categorical(prob)
#         action = dist.sample()
#         log_prob = dist.log_prob(action)
#         return action,log_prob

    def sample_action(self, x):
        with torch.no_grad():  # 在采样时不需要计算梯度
            probs = self.forward(x).squeeze()
            
            # 添加探索机制
            if np.random.random() < 0.5:
                action = np.random.randint(0, 4)
                return (torch.LongTensor([action]), 
                       torch.log(probs[action]))
            
            # 使用策略选择动作
            dist = Categorical(probs)
            action = dist.sample()
            log_prob = dist.log_prob(action)
            
            return action, log_prob

In [ ]:
def discount_rewards(rewards):
    ans = np.zeros_like(rewards)
    adding = 0
    for t in reversed(range(len(rewards))):
        adding = adding * 0.99 + rewards[t]
        ans[t] = adding
    return ans

In [ ]:
model = MLP(4)

In [ ]:
from torch.optim import RMSprop
import torch

In [ ]:
optimizer = RMSprop(model.parameters(), lr=0.001, alpha=0.99, eps=5e-3)

In [ ]:
def sample_action(self, x):
    with torch.no_grad():  # 在采样时不需要计算梯度
        probs = self.forward(x).squeeze()

        # 添加探索机制
        if np.random.random() < 0.0:
            action = np.random.randint(0, 4)
            return (torch.LongTensor([action]), 
                   torch.log(probs[action]))

        # 使用策略选择动作
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)

        return action, log_prob

# 替换方法
import types
model.sample_action = types.MethodType(sample_action, model)

In [ ]:
# env = CliffWalking2x3Env()
observation, _ = env.reset()
episode_number = 0
prev_x = None
xs = []
logps = []
hs = []
actions = []
ys = []
rewards = []
train_time = 0
total_sum = 0
max_times = 0
while True:
    prev_o = observation
    idxs = torch.LongTensor([observation]) # shape (1,)
    action,log_prob = model.sample_action(idxs) # shape 1,  1,
    
    action = action.item() ## 转成数
    logps.append(log_prob)
#     log_prob = log_prob.item()
    
    observation, reward, terminated, truncated, info = env.step(action)  # 更新为新的返回值
    
        # 修改奖励结构
    if reward == -100:  # 掉入悬崖
        reward = -10  # 减小惩罚力度，避免过度规避风险
    elif terminated and observation == 47:  # 到达目标状态（右下角）
        reward = 10  # 给予明显的正向奖励
#     if terminated and observation == 47:  # 到达目标状态（右下角）
#         reward = 10  # 给予明显的正向奖励

#     reward = reward_shaper(prev_o,observation,terminated)
# #     print(action,reward)
#     if terminated:
#         reward = 1000
    
    rewards.append(reward*1.0)
    xs.append(prev_o)
    actions.append(action)
    total_sum += reward
    max_times += 1
    

    if terminated or truncated or max_times > 1000:
#         print(len(actions),actions[:30])
        episode_number += 1
        xs = np.vstack(xs).ravel()
        actions = np.vstack(actions)
        rewards = np.vstack(rewards).ravel()
#         print(rewards)
        rewards = discount_rewards(rewards)
        rewards -= rewards.mean()
        rewards /= (rewards.std()+1e-8)

        rewards = torch.FloatTensor(rewards)
        xs = torch.LongTensor(xs)
        actions = torch.LongTensor(actions) # B,1
        
        ps = model(xs) ### B,4
#         print('ps',ps)
        # 防止概率为0或1导致log计算爆炸
#         logps = torch.cat(logps)
#         print(logps.shape,rewards.shape)
        logps = torch.log(ps.gather(dim=-1,index=actions)+1e-8).squeeze() # B,1
#         print('ps',ps.shape)
#         print('action',actions.shape)
#         print('rewards',rewards)
#         print('logps',logps)
        loss = -(logps * rewards).sum()
        #### 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print('rewards',rewards)
#         print('logps',ps,logps)
        if episode_number % 10 == 0:
            print('total',loss,total_sum,len(logps))
        
        xs = []
        rewards = []
        logps = []
        hs = []
        actions = []
        prev_x = None
        total_sum = 0
        max_times = 0
        observation, _ = env.reset()
        
        

In [ ]:
import time
def visualize_agent(env, model, num_episodes=5):
    """
    渲染显示智能体的行动
    """
    env = gym.make('CliffWalking-v0', render_mode='human')  # 创建可视化环境
#     env = gym.make('CliffWalking-v0')
    for episode in range(num_episodes):
        state_tuple = env.reset()
        state = state_tuple[0] if isinstance(state_tuple, tuple) else state_tuple
        total_reward = 0
        steps = 0
        done = False
        
        print(f"\nEpisode {episode + 1}")
        
        while not done:
            env.render()  # 渲染当前状态
  
            # 使用训练好的策略选择动作
            with torch.no_grad():
                state_tensor = torch.LongTensor([state])
                probs = model(state_tensor)
                action = probs.argmax().item()  # 使用最可能的动作
#                 print(action)
            
            # 执行动作
            step_result = env.step(action)
            if len(step_result) == 4:
                next_state, reward, done, _ = step_result
            else:
                next_state, reward, terminated, truncated, _ = step_result
                done = terminated or truncated
            
            total_reward += reward
            steps += 1
            state = next_state
            
            # 添加小延迟使动作更容易观察
            time.sleep(0.5)
        
        print(f"Episode finished after {steps} steps. Total reward: {total_reward}")
    
    env.close()

# 在主程序最后添加：
if __name__ == "__main__":    
    # 训练完成后显示智能体行动
    print("\nVisualizing trained agent behavior...")
    visualize_agent(env, model)

In [ ]:
env.close()